In [ ]:
%matplotlib inline

## Reading data from text files

Note backslash indicates escape characters - see https://learnpythonthehardway.org/book/ex10.html 

In [ ]:
with open("arts_funding_by_state.txt") as f:
    content = f.readlines()
    for row in content:
        print(row.replace("\n", ""))

We can use the split function to split the lines into lists or a dictionary. In this case the lines are tab-delimited:

In [ ]:
with open("arts_funding_by_state.txt") as f:
    content = f.readlines()
    split_results = []
    for row in content:
        lst = row.replace("\n", "").split("\t")
        split_results.append(lst)
    print(split_results)

In [ ]:
with open("arts_funding_by_state.txt") as f:
    next(f)  # Skip over header line
    content = f.readlines()
    dict_results = dict()
    for row in content:
        lst = row.replace("\n", "").split("\t")
        dict_results[lst[0]] = { "population": lst[1], "funding": lst[2] }
    print(dict_results)

Reading data using the CSV Reader - handles some of the complexities for you. it also supports reading text into a dictionary:

In [ ]:
import csv
with open("arts_funding_by_state.txt") as csv_file:
    rdr = csv.reader(csv_file, delimiter='\t')
    for row in rdr:
        print (row[0], row[1], row[2])

Reading data from text files into a dictionary for each line.

In [ ]:
import csv
with open("arts_funding_by_state.txt") as csv_file:
    rdr = csv.DictReader(csv_file, delimiter='\t')
    for row in rdr:
        print (row)

### Question - Can you write code that reads the contents of the text file using the csvreader, and adds the funding per capita to each list/dictionary as an additional key / value pair?

Here is some sample code for writing a text file.  

In [ ]:
import csv

lines = []
with open("arts_funding_by_state.txt") as csv_file:
    rdr = csv.reader(csv_file, delimiter='\t')
    for row in rdr:
        lines.append([row[0], row[1], row[2], "0"])

with open ('output.txt','a') as output_file:
    for line in lines:
        output_file.write("\t".join(line) + "\n")

print ("done")

### Question: Can you write a program that appends the funding per capita column to the output?

### Reading data from SQL Server

The class machines should be set up with access to the fc_sample database.  Here is sample code that shows how to connect and read data:

In [ ]:
import pyodbc
import matplotlib.pyplot as plt

cnxn = pyodbc.connect("DRIVER={SQL Server};SERVER=172.16.7.121;DATABASE=fc_sample;uid=python_class_user;pwd=[provided in class]")
sql = "select top 10 gm_type, count(*) FROM grantmaker where isnull(gm_type, '') <> '' group by gm_type order by count(*) desc"
cursor = cnxn.cursor()
values = []
labels = []

for row in cursor.execute(sql):
    labels.append(row[0])
    values.append(row[1])
cursor.close()

plt.xticks(range(len(values)), labels)
plt.bar(range(len(values)), height=values)
plt.show()

Some other useful pyodbbc methods - more info here: https://code.google.com/archive/p/pyodbc/wikis/GettingStarted.wiki
* execute - use this to execute any SQL, especially insert/update/delete. Note that .commit() on the database connection needs to be called for the save to be committed and visible to other db users.
* fetchone - useful if you knot that your query only returns a single records.
* fetchall - gets all records into a list.

### Question: can you write python code that queries the fc_sample database to get the number of recipients and grantmakers per state and compare the two in a scatterplot?

***
### Reading data from web services

Python dictionaries can be populated to JSON data from web services.  The json library has two workhorse functions that handle most cases (loads (load string), and dumps (dump string)).

In [ ]:
# JSON - represents data objects (dictionaries) as a string. Can be converted directly to a dictionary in python:
import json as js
json_string = '{ "test1": 987.65, "test2": 124.56 }'
obj = js.loads(json_string)
print (obj["test1"] + obj["test2"])

The requests library (http://docs.python-requests.org/en/master/user/quickstart/) lets us read data from web sites, for example:

In [ ]:
import requests
r = requests.get('http://google.com')
print (r.text)

Reading json data from a web service

In [ ]:
import requests
import json
r = requests.get("http://ws.geonames.net/searchJSON?featureClass=A&featureClass=P&maxRows=10&username=fcgis&q=New York")
obj = json.loads(r.text)
print(json.dumps(obj, indent=2))

Accesing data from new FC API Beta:

In [ ]:
import requests
import base64 as ba
import json as js

s = requests.Session()
s.auth = ('[provided in class]', '[privided in class]')

r = s.get('https://apibeta.foundationcenter.org/v2.0/aggregates/funding')
obj = json.loads(r.text)
print(json.dumps(obj, indent=2))

### Question - Using the documentation at apibeta.foundationcenter.org, can you write a program that gets the top 10 recipients of grants having to do with Museums in 2014, by grant amount?

The PCS code for museums is "SA07", and the list/recipients endpoint should return the top 10 list.

### Can you calculate the total amount given to those 10 recipients?

### Can you create a bar chart of those 10 amounts to get an idea of the distribution?

### Question - combining DB queries with API calls.  Supposed we wanted to get place identifiers mentioned in the grantmaker limitations text so we can put them on a map, for grantmakers located in Brooklyn. You can get this text from the fc_sample database with a query like the following:

```
SELECT 
    a.fcdbwin_gm_key, a.NAME, b.description 
FROM 
    grantmaker a INNER JOIN long_text b ON a.gm_key = b.gm_key 
WHERE 
    b.text_type = 'LM' AND a.state = 'NY' AND a.county = 'Kings' 
```
The api call has this format:

https://apibeta.foundationcenter.org/v2.0/text/entities?text=[text]

Write a program that prints out the geoplace_keys referenced in this text for those grantmakers.

### Can you print out a list of unique ids?

